-------------Загрузка данных----------

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [ ]:
print(data.columns)

In [ ]:
selected_features = [
    ' Flow Duration', 'Total Length of Fwd Packets', 
    ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', 
    ' Fwd IAT Mean', ' Bwd IAT Mean', ' Average Packet Size', ' SYN Flag Count', ' Total Backward Packets',
    ' ACK Flag Count', ' Packet Length Variance', ' Destination Port', ' Protocol'
]


In [ ]:
missing_features = [feature for feature in selected_features if feature not in data.columns]
if missing_features:
    print(f"Missing features in the dataset: {missing_features}")
else:
    print("All selected features are present in the dataset.")


In [ ]:
X = data[selected_features]

y = data[' Label']

--------------Обработка данных-------------

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

In [ ]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)

print(X.isna().sum())

X.dropna(inplace=True)
y = y[X.index]  

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

---------------Создание модели--------------

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(32, input_dim=len(selected_features), activation='relu'))

model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
model.save('modelMLP.h5')

In [ ]:
tensorboard_callback = TensorBoard(log_dir="./logs")
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])

In [ ]:
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

In [ ]:
import time

In [ ]:
single_sample = np.array([X_test[0]]) 

start_time = time.time()
prediction = model.predict(single_sample)
end_time = time.time()

time_taken = end_time - start_time
print(f'Time taken for a single prediction: {time_taken} seconds')

In [ ]:
num_samples = 1000
samples = X_test[:num_samples]  

start_time = time.time()
predictions = model.predict(samples)
end_time = time.time()

time_taken = end_time - start_time
print(f'Time taken for predictions on {num_samples} samples: {time_taken} seconds')


In [ ]:
from memory_profiler import memory_usage

def model_training():
    num_samples = 1000
    samples = X_test[:num_samples] 
    predictions = model.predict(samples)
    pass
mem_usage = memory_usage(model_training)
print(f"Maximum memory usage: {max(mem_usage)} MiB")

In [ ]:
import cProfile

In [ ]:
num_samples = 1000
samples = X_test[:num_samples]  

def run_inference():
    predictions = model.predict(samples)

cProfile.run('run_inference()')

In [ ]:
import timeit
import time

In [ ]:
num_iterations = 100
total_time = 0
total_time_2 = 0

single_sample = np.array([X_test[0]])

for _ in range(num_iterations):
    total_time += timeit.timeit(lambda: model.predict(single_sample), number=1)

average_inference_time = total_time / num_iterations
print(f"Average inference time: {average_inference_time} seconds, totaltame {total_time}")


In [ ]:
single_sample = np.array([X_test[0]])

inference_time = timeit.timeit(lambda: model.predict(single_sample), number=1)
print(f"Inference time: {inference_time} seconds")
